In [1]:
%matplotlib inline

import numpy as np
import random
import pandas as pd
import requests as rq
import sys
import io
import os
from bs4 import BeautifulSoup
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop
from collections import Counter
from datetime import datetime
import keras
import keras.callbacks
from keras.callbacks import TensorBoard
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import Callback
from tensorflow.keras.losses import CategoricalCrossentropy



%load_ext tensorboard

Data Cleaning

In [2]:
##get data
##data is already from 2012-2020
data = pd.read_csv('data_file.csv') # pandas data frame


##preprocessing


##removes 
data = data.loc[(data['desc'].str.startswith('('))]
data = data[data['play_type'] != 'qb_kneel']
data = data[data['play_type'] != 'no_play']
data = data.dropna(subset=['down'])
data = data.dropna(subset=['play_type'])
data = data.dropna(subset=['wp'])
data = data[(data['play_type'] == 'pass') | (data['play_type'] == 'run') | (data['play_type'] == 'field_goal') | (data['play_type'] == 'punt')]
# Look into creating specific year and months, and training and testing on specific years/gmaes. 


# Extract year and month using string slicing
data['year'] = data['game_id'].str[:4].astype(int)
data['month'] = data['game_id'].str[5:7].astype(int)


# Create a new 'date' column with the 1st day of each extracted year and month


# Come back to this when we want to specify split spots  
# def parse_date(x):
#     parts = x.split('_')
#     year = int(parts[0])
#     month = int(parts[1])
#     print(month)
#     return pd.Timestamp(year=int(year), month=month), day=1)

# Apply the function to the 'game_date' column
# data['date'] = data['game_id'].apply(parse_date)
# # Now you can sort by this 'date' column if needed
# data.sort_values(by='date', inplace=True)

# # And use the 'date' column for splitting your data, work with in the future to pick specific dates 
# #Right now not working because the months in the game_id are 01-12, when for pd.TimeStamp they need to be in 1-12. 
# split_date = pd.Timestamp(year=2018, month=12, day=1)  # Replace YYYY and MM with your split year and month
# train_data = data[data['date'] < split_date]
# test_data = data[data['date'] >= split_date]

 
##fixes playtype to be strictly pass or run 
##lables scrambles as passes
data.loc[data['pass']==1,'play_type'] = 'pass'
data.loc[data.rush==1,'play_type'] = 'run'

# columns are the strings 
# things to encode from strings to 1s and zeros
# season_type, 
columnsKeep = ['game_id', 'year','month','season_type','week','posteam','posteam_type','defteam','side_of_field','yardline_100','half_seconds_remaining','home_team','away_team',
               'game_seconds_remaining','game_half','drive','down','time','yrdln','ydstogo','desc','play_type','yards_gained','shotgun','no_huddle','qb_dropback','qb_kneel',
               'qb_spike','qb_scramble','pass_length','run_location','run_gap','home_timeouts_remaining','away_timeouts_remaining','total_home_score','total_away_score',
               'passer_player_name','receiver_player_name','rusher_player_name','play_type_nfl','roof','surface','temp','wind','home_coach','away_coach','game_stadium',
               'success', 'wp']

data = data[columnsKeep]

# print(data.head(5))

data.to_csv('finalized_data.csv', index=False)

/var/folders/ys/6w09q22n4f73ds9jwmgbstjc0000gn/T/ipykernel_1610/2046904662.py:3: DtypeWarning: Columns (45,179,180,182,183,189,190,193,194,197,198,203,204,205,206,209,210,213,214,218,219,220,222,224,226,233,234,235,236,237,238,248,249,253,254,255,260,262,263,266,267,268,269,283,284,294,295,301) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data_file.csv') # pandas data frame


In [51]:
print(len(data))

# Count occurrences of each play type
runCount = (data['play_type'] == 'run').sum()
passCount = (data['play_type'] == 'pass').sum()
fgCount = (data['play_type'] == 'field_goal').sum()
puntCount = (data['play_type'] == 'punt').sum()


print("Number of 'run' plays:", runCount)
print("Number of 'pass' plays:", passCount)
print("Number of 'fg' plays:", fgCount)
print("Number of 'punt' plays:", puntCount)



334503
Number of 'run' plays: 117123
Number of 'pass' plays: 186390
Number of 'fg' plays: 9285
Number of 'punt' plays: 21705


In [19]:
##check for nan values in data
for index, row in data.iterrows():
    if pd.isnull(row['play_type']):  # Check if the value in the 'down' column is NaN
        print(row['desc'])  # Print the value in the 'desc' column if 'down' is NaN

# wind and stadium had a lot of null 
## check for missing values in data
selected_features = ['down', 'drive', 'ydstogo', 'yardline_100', 'half_seconds_remaining', 'game_seconds_remaining','wp','week','yards_gained']
missing_values_features = data[selected_features].isnull().sum()
print("Missing values in selected features:")
print(missing_values_features)

# Check for missing values in the target
missing_values_target = data['play_type'].isnull().sum()
print("\nMissing values in target (play_type):", missing_values_target)


Missing values in selected features:
down                      0
drive                     0
ydstogo                   0
yardline_100              0
half_seconds_remaining    0
game_seconds_remaining    0
wp                        0
week                      0
yards_gained              0
dtype: int64

Missing values in target (play_type): 0


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

features = data[['down','drive', 'ydstogo','yardline_100', 'half_seconds_remaining', 'game_seconds_remaining', 'wp']]
target = data['play_type'].map({'pass': 0, 'run': 1, 'field_goal':2,'punt':3})

# Initialize the scaler 
scaler = StandardScaler() 

# Fit the scaler on the features and transform them 
features_scaled = scaler.fit_transform(features)

# Reshaping input data for LSTM [samples, time steps, features]
# Here, we assume each sample is a single time step
features_scaled = features_scaled.reshape((features_scaled.shape[0], 1, features_scaled.shape[1]))

# Talk to Nicolai and Ulrich if they think this is a good idea to split it up into drives as our time stamp. 
# 1 indicates that each sample consists of 1 timestep. This is a key detail: 
# it means you are treating each play as a separate, single-timestep sequence. T
# This is a simplification, as ideally, plays within a single drive might be treated as sequences to capture their temporal dependencies.

X_train, X_test, y_train, y_test = train_test_split(features_scaled, target, test_size=0.2, shuffle = False)




In [20]:
print(target.unique())


[1 0 3 2]


In [5]:
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    LSTM(32,return_sequences=False),
    Dense(4, activation='sigmoid')
])

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=Adam(learning_rate=0.001),
    metrics=['accuracy']
)

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.1)


test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc:.3f}, Test Loss: {test_loss:.3f}')

Epoch 1/10


2024-04-23 12:52:18.550924: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-23 12:52:18.552044: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-23 12:52:18.552383: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

3717/3764 [============================>.] - ETA: 0s - loss: 0.5854 - accuracy: 0.6973

2024-04-23 12:52:23.056192: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-23 12:52:23.056855: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-23 12:52:23.057267: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

3764/3764 [==============================] - 5s 988us/step - loss: 0.5848 - accuracy: 0.6974 - val_loss: 0.5413 - val_accuracy: 0.7089
Epoch 2/10
3764/3764 [==============================] - 4s 942us/step - loss: 0.5343 - accuracy: 0.7142 - val_loss: 0.5278 - val_accuracy: 0.7181
Epoch 3/10
3764/3764 [==============================] - 3s 910us/step - loss: 0.5273 - accuracy: 0.7169 - val_loss: 0.5278 - val_accuracy: 0.7182
Epoch 4/10
3764/3764 [==============================] - 3s 909us/step - loss: 0.5230 - accuracy: 0.7196 - val_loss: 0.5273 - val_accuracy: 0.7185
Epoch 5/10
3764/3764 [==============================] - 3s 907us/step - loss: 0.5204 - accuracy: 0.7208 - val_loss: 0.5190 - val_accuracy: 0.7230
Epoch 6/10
3764/3764 [==============================] - 4s 944us/step - loss: 0.5182 - accuracy: 0.7220 - val_loss: 0.5191 - val_accuracy: 0.7235
Epoch 7/10
3764/3764 [==============================] - 3s 900us/step - loss: 0.5167 - accuracy: 0.7219 - val_loss: 0.5190 - val_accura

In [6]:

predictions = model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)

from sklearn.metrics import classification_report, confusion_matrix

print("Classification Report:")
print(classification_report(y_test, predicted_classes))

print("Confusion Matrix:")
print(confusion_matrix(y_test, predicted_classes))


   1/2091 [..............................] - ETA: 8:39

2024-04-23 12:52:59.251153: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-23 12:52:59.251750: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-23 12:52:59.252200: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2091/2091 [==============================] - 1s 293us/step
Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.74      0.76     37737
           1       0.62      0.64      0.63     23520
           2       0.87      0.84      0.85      1858
           3       0.91      0.99      0.94      3786

    accuracy                           0.72     66901
   macro avg       0.79      0.80      0.79     66901
weighted avg       0.72      0.72      0.72     66901

Confusion Matrix:
[[28101  9280   153   203]
 [ 8313 15002    68   137]
 [  210    46  1557    45]
 [   38     5    13  3730]]


In [17]:
dummies = pd.get_dummies(data['play_type'], prefix='play_type')

df = pd.concat([data, dummies], axis=1)






In [18]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))

corr_matrix = df.corr()

sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', cbar=True)

plt.show()

ValueError: could not convert string to float: '2012_01_ATL_KC'

<Figure size 800x600 with 0 Axes>

In [26]:
# length of X train, and nulls in the x,train, x_test, y_train, y_test 
print(len(X_train))
nan_indices_X = np.isnan(X_train)
nan_count_X = np.sum(nan_indices_X)
print(nan_count_X)

nan_indices_X_test = np.isnan(X_test)
nan_count_X_test = np.sum(nan_indices_X_test)
print(nan_count_X_test)

nan_indices_y = np.isnan(y_train)
nan_count_y = np.sum(nan_indices_y)
print(nan_count_y)

nan_indices_Y_test = np.isnan(y_test)
nan_count_Y_test = np.sum(nan_indices_Y_test)
print(nan_count_Y_test)

print("Shape of X_train:", X_train.shape)

267602
0
0
0
0
Shape of X_train: (267602, 1, 7)
